<a href="https://colab.research.google.com/github/adw1029/CSCI_544/blob/main/HW1_CSCI544.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip setuptools wheel

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=184b231779ec7d6e57313837ac1982281375c377c5a0bd4ef31bfdd493e228db
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


## Read Data

In [3]:
import os
current_directory = os.getcwd()

In [4]:
print("Current Directory:", current_directory)


Current Directory: /content


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
google_drive_directory = '/content/drive/My Drive/CSCI_544/hw1'
os.chdir(google_drive_directory)

In [7]:
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content/drive/My Drive/CSCI_544/hw1


In [8]:
gz_file_path = 'amz_data.gz'

## Keep Reviews and Ratings

In [9]:
file_path = 'amz_data.tsv'
data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])

<ipython-input-9-04b759942888>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])


In [10]:
print(data)

        star_rating                                        review_body
0                 5                                     Great product.
1                 5  What's to say about this commodity item except...
2                 5    Haven't used yet, but I am sure I will like it.
3                 1  Although this was labeled as &#34;new&#34; the...
4                 4                    Gorgeous colors and easy to use
...             ...                                                ...
2640347           4  I can't live anymore whithout my Palm III. But...
2640348           4  Although the Palm Pilot is thin and compact it...
2640349           4  This book had a lot of great content without b...
2640350           5  I am teaching a course in Excel and am using t...
2640351           5  A very comprehensive layout of exactly how Vis...

[2640352 rows x 2 columns]


Separate out review and rating, dropped null values

In [11]:
df_RevRat = data[data['star_rating'] != 3]
df_RevRat = data.dropna(subset=['star_rating', 'review_body'])

In [12]:
df_RevRat

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [13]:
df_RevRat['star_rating'] = pd.to_numeric(df_RevRat['star_rating'], errors='coerce')


<ipython-input-13-835b95727621>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['star_rating'] = pd.to_numeric(df_RevRat['star_rating'], errors='coerce')


In [14]:
df_RevRat['star_rating'].dtypes

dtype('float64')

In [15]:
df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)

<ipython-input-15-940d36b9d541>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)


In [16]:
df_RevRat['Sentiment']

0          1
1          1
2          1
3          0
4          1
          ..
2640347    1
2640348    1
2640349    1
2640350    1
2640351    1
Name: Sentiment, Length: 2640255, dtype: int64

check if there's null values

In [17]:
contains_null = df_RevRat['star_rating'].isnull().any()
contains_null

False

## 20000 samples from positive (100000) and negative (100000) reviews

In [18]:
pos_rev = df_RevRat[df_RevRat['Sentiment'] == 1].sample(100000, random_state=42)
neg_rev = df_RevRat[df_RevRat['Sentiment'] == 0].sample(100000, random_state=42)
df_small = pd.concat([pos_rev, neg_rev])

In [47]:
df_small.sample(3)

,star_rating,review_body,Sentiment
677101,5.0,initially purchased set printer cartridge daug...,1
1823428,4.0,using pixma cannot tell difference came printe...,1
788756,1.0,hate write bad review seriously bad purchase c...,0


# Data Cleaning



# avg review length before cleaning

In [20]:
avglen_before = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_before)


Average text length: 315.554375


reviews to lower case

In [21]:
df_small['review_body'] = df_small['review_body'].str.lower()


remove URL and HTML

In [22]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')
html_pattern = re.compile('<.*?>')

In [23]:
def remove_html(text):
    return html_pattern.sub(r'', text)

# Function to remove URLs
def remove_urls(text):
    return url_pattern.sub(r'', text)

# Apply the functions to the review column
df_small['review_body'] = df_small['review_body'].apply(remove_html).apply(remove_urls)

remove non-alpha

In [24]:
df_small['review_body'] = df_small['review_body'].str.replace('[^a-zA-Z]', ' ', regex=True)

remove extra spaces

In [25]:
df_small['review_body'] = df_small['review_body'].str.replace(r'\s+', ' ', regex=True)

contractions

In [26]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.4 MB/s eta 0:00:00


In [27]:
import contractions


In [28]:
df_small['review_body'] = df_small['review_body'].apply(contractions.fix)

#average review length after cleaning

In [29]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 300.445865


# Pre-processing

In [30]:
from nltk.corpus import stopwords

### 3 sample reviews before pre-processing / Avg length

In [31]:
print(df_small['review_body'].head(3))

386895     i reviewed the packaging previously and gave s...
2514581    i got tired of searching my place for envelope...
785896               the best faber castell never dissapoint
Name: review_body, dtype: object


In [32]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 300.445865


### remove the stop words

In [33]:
nltk.download('stopwords')

stp_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [34]:
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stp_words]))


## perform lemmatization  

In [35]:
from nltk.stem import WordNetLemmatizer

In [36]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [37]:
lemmatizer = WordNetLemmatizer()
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


In [38]:
print(df_small['review_body'].head(3))

386895     reviewed packaging previously gave star replac...
2514581    got tired searching place envelope decided buy...
785896                   best faber castell never dissapoint
Name: review_body, dtype: object


In [39]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 184.625575


# TF-IDF Feature Extraction

#split testing (20%) and training (80%) set


In [118]:
vectorizer = TfidfVectorizer(max_features = 10000)
tfidf_mat = vectorizer.fit_transform(df_small['review_body'])

In [119]:
tfidf_mat

<200000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4508748 stored elements in Compressed Sparse Row format>

# Perceptron

In [120]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [121]:
X = tfidf_mat
y = df_small['Sentiment']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
perceptron = Perceptron(random_state= 42).fit(X_train, y_train)

In [124]:
y_train_pred = perceptron.predict(X_train)
y_test_pred = perceptron.predict(X_test)


In [125]:
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

In [126]:
print("Training Metrics:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1 Score:", train_f1)

print("\nTesting Metrics:")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1 Score:", test_f1)

Training Metrics:
Accuracy: 0.81854375
Precision: 0.831637432988081
Recall: 0.7988426012723887
F1 Score: 0.8149102059811678

Testing Metrics:
Accuracy: 0.798725
Precision: 0.8130767617449665
Recall: 0.7756214675136298
F1 Score: 0.7939075899142456


# SVM

# Logistic Regression

# Naive Bayes